In [1]:
import visualize
import pandas as pd
from utils.utils import add_color_to_tags, extract_parts_0, extract_parts_1
import argparse


In [3]:
def create_highlight_html(questions, answers):
    html_content = """
    <!DOCTYPE html>
    <html lang="en">
    <head>
        <meta charset="UTF-8">
        <title>Question and Answer Highlights</title>
        <style>
            body {
                font-family: Arial, sans-serif;
                margin: 20px;
            }
            .highlight {
                background-color: #FFFF00; /* Yellow background for visibility */
                font-weight: bold; /* Bold text for emphasis */
            }
            .container {
                display: flex;
                justify-content: space-between;
                margin-bottom: 20px;
            }
            .question, .answer {
                flex: 1;
                padding: 10px;
            }
        </style>
    </head>
    <body>
    """
    for i, (question, answer) in enumerate(zip(questions, answers)):
        try:
            reformulated_question, extracted_answer = extract_parts_1(answer)
        except:
            print("Can not extract parts...")
            continue
        
        highlighted_answer = add_color_to_tags(extracted_answer)
        highlighted_question = add_color_to_tags(reformulated_question)
        
        html_content += f"<div class='container'>"
        html_content += f"<div class='question'><strong>Question:</strong> {question}</div>"
        html_content += f"<div class='answer'><strong>Reformulated Question:</strong> {highlighted_question}<br><br><strong>Answer:</strong>{highlighted_answer}</div>"
        html_content += "</div>\n"

    # Close the HTML tags
    html_content += """
    </body>
    </html>
    """

    return html_content

In [11]:
questions = ["Question: Did Donald Trump come up with the idea for the New York Harbor?"]
answers = ["Reformatted Question: Jim spends <a>2 hours watching TV</a> and then decides to go to bed and <b>reads for half as long</b>. He does this <c>3 times a week</c>. How many hours does he spend on TV and reading in <d>4 weeks</d>? Answer: Each time Jim watches TV, he spends <a>2 hours</a>. He reads for half as long, which is <a>2 hours</a> * <b>1/2</b> = 1 hour. So, each time, he spends 2 hours on TV and 1 hour reading, totaling 3 hours per session. He does this <c>3 times a week</c>, so in one week he spends 3 hours/session * 3 sessions = 9 hours. Over <d>4 weeks</d>, he spends 9 hours/week * 4 weeks = 36 hours. The answer is 36."]
html_content = create_highlight_html(questions, answers)
with open("mini2.html", "w") as file:
    file.write(html_content)

In [14]:
question = "Question: How many cheese balls are in a 35oz barrel if the 24oz barrel has 60 servings and each serving has 12 cheese balls?"
fewshot_prompt = f"""Question: For every 12 cans you recycle, you receive $0.50, and for every 5 kilograms of newspapers, you receive $1.50. If your family collected 144 cans and 20 kilograms of newspapers, how much money would you receive?
Reformatted Question: For <a>every 12 cans you recycle</a>, <b>you receive $0.50</b>, and for <c>every 5 kilograms of newspapers</c>, <d>you receive $1.50</d>. If your family collected <e>144 cans</e> and <f>20 kilograms of newspapers</f>, how much money would you receive?
Answer: There are <e>144</e>/<a>12</a> = 12 sets of 12 cans that the family collected. So, the family would receive <b>$0.50</b> x 12 = $6 for the cans. There are <f>20</f>/5 = 4 sets of <c>5 kilograms of newspapers</c> that the family collected. So, the family would receive <d>$1.50</d> x 4 = $6 for the newspapers. Therefore, the family would receive a total of $6 + $6 = $12.
The answer is 12.

Question: Betty picked 16 strawberries. Matthew picked 20 more strawberries than Betty and twice as many as Natalie. They used their strawberries to make jam. One jar of jam used 7 strawberries and they sold each jar at $4. How much money were they able to make from the strawberries they picked?
Reformatted Question: <a>Betty picked 16 strawberries</a>. <b>Matthew picked 20 more strawberries than Betty</b> and <c>twice as many as Natalie</c>. They used their strawberries to make jam. <d>One jar of jam used 7 strawberries</d> and they <e>sold each jar at $4</e>. How much money were they able to make from the strawberries they picked?
Answer: <a>Matthew picked 16</a> + <b>20</b> = 36 strawberries. <c>Natalie picked 36/2 = 18 strawberries </c>. All together, they have <a>16</a> + 36 + 18 = 70 strawberries. They can make <d>70/7 = 10 jars of strawberries </d>. They earn <e>10 x $4 = $40 from the strawberries they picked </e>.
The answer is 40.

Question: Jack has a stack of books that is 12 inches thick. He knows from experience that 80 pages is one inch thick. If he has 6 books, how many pages is each one on average?
Reformatted Question: Jack has a stack of books that is <b>12 inches thick</b>. He knows from experience that <a>80 pages is one inch thick</a>. If he has <c>6 books</c>, how many pages is each one on average?
Answer: There are 960 pages because <a>80</a> x <b>12</b> = 960. Each book is 160 pages because 960 / <c>6</c> = 160.
The answer is 160.

Question: James dumps his whole collection of 500 Legos on the floor and starts building a castle out of them. He uses half the pieces before finishing and is told to put the rest away. He puts all of the leftover pieces back in the box they came from, except for 5 missing pieces that he can't find. How many Legos are in the box at the end?
Reformatted Question: James <a>dumps his whole collection of 500 Legos</a> on the floor and starts building a castle out of them. He <b>uses half the pieces</b> before finishing and is told to put the rest away. He puts all of the leftover pieces back in the box they came from, <c>except for 5 missing pieces</c> that he can't find. How many Legos are in the box at the end?
Answer: James starts with <a>500 Legos</a> and <b>uses half of them</b>, leaving 500/2=250 Legos unused. He puts those unused Legos away but since he's missing <c>5</c> he only puts 250-5=245 Legos away.
The answer is 245.

Question: Ines had $20 in her purse. She bought 3 pounds of peaches, which are $2 per pound at the local farmers’ market. How much did she have left?
Reformatted Question: Ines <c>had $20 in her purse</c>. She bought <a>3 pounds of peaches</a>, which are <b>$2 per pound</b> at the local farmers’ market. How much did she have left?
Answer: Ines bought <a>3 pounds of peaches</a> for <a>3/peach</a> * <b>$2/pound</b> = $6. Ines has <c>$20</c> - $6 = $14 left.
The answer is 14.

Question: Aaron pays his actuary membership fees each year. The membership fee increases yearly by $10. If he pays $80 in the first year, how much does his membership cost, in dollars, in the sixth year?
Reformatted Question: Aaron pays his actuary membership fees each year. The <a>membership fee increases yearly by $10</a>. If he pays $80 in the first year, how much does his membership cost, in dollars, in the sixth year?
Answer: In year 2, he pays 80+<a>10</a>=$90. In year 3, he pays 90+<a>10</a>=$100. In year 4, he pays 100+<a>10</a>=$110. In year 5, he pays 110+<a>10</a>=$120. In year 6, he pays 120+<a>10</a>=$130.
The answer is 130.

Question: Joseph invested $1000 into a hedge fund. The fund promised a yearly interest rate of 10%. If he deposited an additional $100 every month into the account to add to his initial investment of $1000, how much money will he have in the fund after two years?
Reformatted Question: <a>Joseph invested $1000 into a hedge fund</a>. The fund promised a <c>yearly interest rate of 10%</c>. If he deposited an additional <b>$100 every month</b> into the account to add to his initial investment of $1000, how much money will he have in the fund after two years?
Answer: For the first year, Joseph will have invested <a>$1000</a> + (<b>$100</b> * 12) = $2200. The interest calculated for the first year will be $2200 * <c>10%</c> = $220. The total value of the investment for the first year will be $2200 + $220 = $2420. For the second year, the total invested will be $2420 + (<b>$100</b> * 12) = $3620. The interest calculated after the second year will be $3620 * <c>10%</c> = $362. Therefore, Joseph's investment in the hedge fund will be worth $3620 + $362 = $3982.
The answer is 3982.

Question: The price of buying a wooden toy at the new Craftee And Best store is $20, and the cost of buying a hat is $10. If Kendra went to the shop with a $100 bill and bought two wooden toys and three hats, calculate the change she received.
Reformatted Question: The price of buying <b>a wooden toy at the new Craftee And Best store is $20</b>, and <c>the cost of buying a hat is $10</c>. If Kendra <e>went to the shop with a $100 bill</e> and bought <a>two wooden toys</a> and <d>three hats</d>, calculate the change she received.
Answer: When Kendra bought <a>2 toys</a>, she paid 2*<b>$20</b> = $40. Since the price of a hat is <c>$10</c>, when Kendra bought <d>3 hats</d>, she paid 3*<c>$10</c> = $30. The total costs for the hats and wooden toys Kendra bought is $40+$30 = $70. From the <e>$100 bill</e>, Kendra received change worth $100-$70 = $30.
The answer is 30.

-------------------------------------------

I want you to answer the following question using the same format as the examples provided previously. Your explanation should contain references referring back to the information in the question. To do that, first, re-generate the question with proper tags and then generate your answers.

{question}
"""

print(fewshot_prompt)

Question: For every 12 cans you recycle, you receive $0.50, and for every 5 kilograms of newspapers, you receive $1.50. If your family collected 144 cans and 20 kilograms of newspapers, how much money would you receive?
Reformatted Question: For <a>every 12 cans you recycle</a>, <b>you receive $0.50</b>, and for <c>every 5 kilograms of newspapers</c>, <d>you receive $1.50</d>. If your family collected <e>144 cans</e> and <f>20 kilograms of newspapers</f>, how much money would you receive?
Answer: There are <e>144</e>/<a>12</a> = 12 sets of 12 cans that the family collected. So, the family would receive <b>$0.50</b> x 12 = $6 for the cans. There are <f>20</f>/5 = 4 sets of <c>5 kilograms of newspapers</c> that the family collected. So, the family would receive <d>$1.50</d> x 4 = $6 for the newspapers. Therefore, the family would receive a total of $6 + $6 = $12.
The answer is 12.

Question: Betty picked 16 strawberries. Matthew picked 20 more strawberries than Betty and twice as many as

In [5]:
import requests
import json

# Set up the API endpoint
url = "http://localhost:11434/api/generate"

# Set up the prompt and model parameters
data = {
    "model": "llama3.1",
    "prompt": f"{fewshot_prompt}",
    "stream": False
}

# Make the API call
response = requests.post(url, json=data)

# Parse the response
if response.status_code == 200:
    result = response.json()
    generated_text = result['response']
    print(generated_text)
else:
    print(f"Error: {response.status_code}")
    print(response.text)

Why did the programmer quit his job?

Because he didn't get arrays of fruit from his work. He was fed up with all the null pointers in his career.
